<a href="https://colab.research.google.com/github/apschlissel/w266-final-project/blob/main/T5_translation_kevin_edit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# T5 Translation Model

Summary:
* Replace slang text in reddit posts with de-slanged text
* Manually check replaced text to ensure posts make sense
* Train a T5 model on checked de-slanged text

In [ ]:
!pip install -q transformers

In [ ]:
!pip install simpletransformers

     |████████████████████████████████| 249 kB 5.2 MB/s 
     |████████████████████████████████| 10.1 MB 50.8 MB/s 
     |████████████████████████████████| 1.7 MB 50.8 MB/s 
     |████████████████████████████████| 1.2 MB 43.0 MB/s 
     |████████████████████████████████| 325 kB 31.4 MB/s 
     |████████████████████████████████| 43 kB 1.9 MB/s 
     |████████████████████████████████| 144 kB 59.4 MB/s 
     |████████████████████████████████| 181 kB 59.6 MB/s 
     |████████████████████████████████| 63 kB 1.5 MB/s 
     |████████████████████████████████| 1.1 MB 52.9 MB/s 
     |████████████████████████████████| 212 kB 62.0 MB/s 
     |████████████████████████████████| 134 kB 62.2 MB/s 
     |████████████████████████████████| 127 kB 67.3 MB/s 
     |████████████████████████████████| 144 kB 70.9 MB/s 
     |████████████████████████████████| 271 kB 45.6 MB/s 
     |████████████████████████████████| 94 kB 3.2 MB/s 
     |████████████████████████████████| 111 kB 71.2 MB/s 
     |██████████████

In [ ]:
from __future__ import print_function
import ipywidgets as widgets
from transformers import pipeline
from simpletransformers.t5 import T5Model, T5Args
import pandas as pd
import logging
import numpy as np
import torch
from tqdm.notebook import tqdm
from transformers import BertTokenizer
from torch.utils.data import TensorDataset
from transformers import BertForSequenceClassification
import json
import re
import random
import math
from bs4 import BeautifulSoup
# Pull reddit data from reddit api
import requests
pd.options.display.max_colwidth = 1000
pd.set_option('display.max_rows', 100)

## Pull Reddit Data

In [ ]:
# note that CLIENT_ID refers to 'personal use script' and SECRET_TOKEN to 'token'
auth = requests.auth.HTTPBasicAuth('pigKA_TKnDkXcatEGcbo8g', 'nawGKK2MfPtC6vKz8TjaNEnmYfAggA')

# here we pass our login method (password), username, and password
data = {'grant_type': 'password',
        'username': 'Katsuuu100',
        'password': 'Testing159753'}

# setup our header info, which gives reddit a brief description of our app
headers = {'User-Agent': 'MyBot/0.0.1'}

# send our request for an OAuth token
res = requests.post('https://www.reddit.com/api/v1/access_token',
                    auth=auth, data=data, headers=headers)

# convert response to JSON and pull access_token value
TOKEN = res.json()['access_token']

# add authorization to our headers dictionary
headers = {**headers, **{'Authorization': f"bearer {TOKEN}"}}

# while the token is valid (~2 hours) we just add headers=headers to our requests
requests.get('https://oauth.reddit.com/api/v1/me', headers=headers)

<Response [200]>

In [ ]:
# HTML web scraper, scrape top subreddits, SFW only.
# 
# Source: https://realpython.com/beautiful-soup-web-scraper-python/
# Source: https://stackoverflow.com/questions/40210093/how-do-i-scrape-only-div-class-quotetext-from-a-website-using-python


URL = "http://redditlist.com/sfw/"
page = requests.get(URL)

soup = BeautifulSoup(page.content, "html.parser")
# print(soup.prettify())
job_elements = soup.find_all("div", class_="listing-item")

# print(job_elements[0])

# Filter to only the Top 125 Most subscribed subreddits
job_elements = job_elements[125:250]

master_subreddit_list = []
# Find subreddit names
for job_element in job_elements:
  links = job_element.find_all("a")
  for link in links:
    print(link.text.strip())
    master_subreddit_list.append(link.text.strip())

announcements
funny
AskReddit
gaming
aww
Music
pics
worldnews
movies
science
todayilearned
videos
news
Showerthoughts
Jokes
food
askscience
IAmA
EarthPorn
gifs
nottheonion
books
DIY
explainlikeimfive
Art
LifeProTips
space
sports
mildlyinteresting
Documentaries
gadgets
memes
tifu
photoshopbattles
UpliftingNews
GetMotivated
dataisbeautiful
listentothis
history
philosophy
television
InternetIsBeautiful
Futurology
WritingPrompts
OldSchoolCool
personalfinance
nosleep
creepy
TwoXChromosomes
wallstreetbets
technology
wholesomememes
AdviceAnimals
interestingasfuck
Fitness
politics
WTF
lifehacks
oddlysatisfying
relationship_advice
NatureIsFuckingLit
Minecraft
travel
facepalm
Whatcouldgowrong
nextfuckinglevel
pcmasterrace
leagueoflegends
BlackPeopleTwitter
me_irl
Unexpected
dankmemes
bestof
dadjokes
buildapc
Tinder
PS4
MadeMeSmile
AnimalsBeingBros
Damnthatsinteresting
tattoos
CryptoCurrency
AnimalsBeingJerks
photography
nba
AnimalsBeingDerps
gardening
BikiniBottomTwitter
trippinthroughtime
Watch

In [ ]:
# Pick 5 random subreddits
five_random_subreddits = random.choices(master_subreddit_list, k=5)
print(five_random_subreddits)

['videos', 'CryptoCurrency', 'Tinder', 'Tinder', 'WatchPeopleDieInside']


In [ ]:
# Source: https://towardsdatascience.com/how-to-use-the-reddit-api-in-python-5e05ddfd1e5c
# Source: https://pynative.com/python-random-choice/
# Pull from 5 classes. 5 classes = 5 subreddits.

my_list_of_dictionaries = []
total = 0
# Target Count for test set = 100.
n = int(math.ceil(100/0.20/25))

# url_list_check = [f"https://oauth.reddit.com/r/{five_random_subreddits[0]}/new/",
#           f"https://oauth.reddit.com/r/{five_random_subreddits[1]}/new/",
#           f"https://oauth.reddit.com/r/{five_random_subreddits[2]}/new/",
#           f"https://oauth.reddit.com/r/{five_random_subreddits[3]}/new/",
#           f"https://oauth.reddit.com/r/{five_random_subreddits[4]}/new/"
#           ]

url_list_check = [f"https://oauth.reddit.com/r/wallstreetbets/new/",
          f"https://oauth.reddit.com/r/teenagers/new/",
          f"https://oauth.reddit.com/r/copypasta/new/",
          f"https://oauth.reddit.com/r/genz/new/",
          f"https://oauth.reddit.com/r/unpopularopinion/new/",
          # f"https://oauth.reddit.com/r/frat/new/"
          ]

for i in range(len(url_list_check)):
    
  # print(url_list_check[i])
  res_check = requests.get(url_list_check[i],
                    headers=headers,
                    params={"limit": "1"})
  
  # print(res_check)
  # print(json.dumps(res_check.json()["data"]["children"][0]["data"]["name"], indent=4))
  name = res_check.json()["data"]["children"][0]["data"]["name"]
  page_count = 25
  
  for j in range(n):

    # url_list = [f"https://oauth.reddit.com/r/{five_random_subreddits[0]}/new/?count={page_count}&after={name}",
    #         f"https://oauth.reddit.com/r/{five_random_subreddits[1]}/new/?count={page_count}&after={name}",
    #         f"https://oauth.reddit.com/r/{five_random_subreddits[2]}/new/?count={page_count}&after={name}",
    #         f"https://oauth.reddit.com/r/{five_random_subreddits[3]}/new/?count={page_count}&after={name}",
    #         f"https://oauth.reddit.com/r/{five_random_subreddits[4]}/new/?count={page_count}&after={name}"
    #         ]

    url_list = [f"https://oauth.reddit.com/r/wallstreetbets/new/?count={page_count}&after={name}",
            f"https://oauth.reddit.com/r/teenagers/new/?count={page_count}&after={name}",
            f"https://oauth.reddit.com/r/copypasta/new/?count={page_count}&after={name}",
            f"https://oauth.reddit.com/r/genz/new/?count={page_count}&after={name}",
            f"https://oauth.reddit.com/r/unpopularopinion/new/?count={page_count}&after={name}",
            # f"https://oauth.reddit.com/r/frat/new/?count={page_count}&after={name}"
            ]
    
    print("Page Count:", page_count)
    print("Name:", name)
    print("Url:", url_list[i])
    
    res = requests.get(url_list[i],
                    headers=headers)
                    # params={"limit": "100"})

    reddit_dictionary = res.json()

    for k in range(len(reddit_dictionary["data"]["children"])):
      my_dictionary = {}
      my_dictionary["subreddit"] = reddit_dictionary["data"]["children"][k]["data"]["subreddit"]
      my_dictionary["text"] = reddit_dictionary["data"]["children"][k]["data"]["selftext"]
      # If a reddit post is has no body text:
      if my_dictionary["text"] == "":
        # Replace with title of reddit post.
        my_dictionary["text"] = reddit_dictionary["data"]["children"][k]["data"]["title"]
      
      print(my_dictionary["text"])
      # my_dictionary["title"] = reddit_dictionary["data"]["children"][k]["data"]["title"]
      # my_dictionary["url"] = reddit_dictionary["data"]["children"][k]["data"]["url"]
      # print(reddit_dictionary["data"]["children"][k]["data"]["subreddit"])
      # print(reddit_dictionary["data"]["children"][k]["data"]["selftext"])
      # print(reddit_dictionary["data"]["children"][k]["data"]["url"])
      my_list_of_dictionaries.append(my_dictionary)
      total += 1
      name = reddit_dictionary["data"]["children"][k]["data"]["name"]
    # print(json.dumps(my_list_of_dictionaries, indent=4, sort_keys=False))
    
    page_count += 25

print("Total gathered:", total)

Streaming output truncated to the last 5000 lines.

Melatonin is synthesized from The Amino Acid Tryptophan which is one of the primary 21 Amino Acid building blocks of protein in our body, taken; 
*3 hours before sleep time 
is found to help 
*Regulate Sleep Cycle, hence aiding our 
*natural circadian rhythm 
to sleep faster ON TIME And to wake up ON TIME, without grogginess you get from medicines!
So The bare-bone take-home message is to take Melatonin in Consort &amp; in Harmony with our 
Sleep Timing Pattern in order for it to Regulate our 
Natural Circadian Rhythm! About 3 Hours before our anticipated sleep time!
Amen!?
Please help my tummy hurts and my throat feels funny.

I dont know what it is but I believe it may be from the 2 gallons of TwinkCream I ate a few hours ago
Fun fact: the reasons americans call football "soccer" is due to an 1878 lawsuit. The original forme of the american game was invented by medical students and used a severed human foot as the "ball", with the l

In [ ]:
# Source: https://github.com/susanli2016/NLP-with-Python/blob/master/Text_Classification_With_BERT.ipynb
# Convert list of dictionaries into pandas df
df = pd.DataFrame(my_list_of_dictionaries)
df.head()

In [ ]:
df['subreddit'].value_counts()

In [ ]:
possible_labels = df.subreddit.unique()

label_dict = {}
for index, possible_label in enumerate(possible_labels):
    label_dict[possible_label] = index
label_dict

In [ ]:
df['label'] = df.subreddit.replace(label_dict)
df.head()

In [ ]:
#combine title & text to make one column
#df['title_and_text'] = df['title'] + ' ' +  df['text']
#df['title_and_text'].head()

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(df.index.values, 
                                                  df.label.values, 
                                                  test_size=0.20, 
                                                  random_state=42, 
                                                  stratify=df.label.values)

In [ ]:
df['data_type'] = ['not_set']*df.shape[0]

df.loc[X_train, 'data_type'] = 'train'
df.loc[X_val, 'data_type'] = 'val'

df.groupby(['subreddit', 'label', 'data_type']).count()

text
subreddit        label data_type      
GenZ             3     train       400
                       val         100
copypasta        2     train       400
                       val         100
teenagers        1     train       400
                       val         100
unpopularopinion 4     train       400
                       val         100
wallstreetbets   0     train       400
                       val         100

## Load Slangit Data

Slangit is a direct translation of slang data

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
slang = pd.read_csv('/content/gdrive/MyDrive/W266/final_project/slangit_kevin_edit.csv')
slang.head(20)

In [ ]:
slangit_dict = slang.set_index('Slang Term').to_dict()
slangit_dict = slangit_dict['Meaning']

In [ ]:
keys_values = slangit_dict.items()
slangit_dict = {str(key): str(value) for key, value in keys_values}
# slangit_dict =  {k.lower(): v for k, v in slangit_dict.items()}

In [ ]:
# Source: https://stackoverflow.com/questions/52976850/case-insensitive-regex-returning-original-pattern
# Source: https://stackoverflow.com/questions/554013/regular-expression-to-split-on-spaces-unless-in-quotes
def slang_lookup(text, dictionary):
    
    pattern = re.compile(r'(?<!\w)(' + '|'.join(re.escape(key) for key in slangit_dict.keys()) + r')(?!\w)')
    result = pattern.sub(lambda x: dictionary[x.group()], text)

    return result

In [ ]:
my_text = 'I watched the UNC game at a bar b/c YOLO, FTW'

print(slang_lookup(my_text, slangit_dict))

## Apply Slangit regex replace to reddit data

In [ ]:
df_train = df[df.index.isin(X_train)]
len(df_train)

In [ ]:
df_train['text_deslanged'] = df_train['text'].apply(lambda x: slang_lookup(x, slangit_dict))
df_train['text_deslanged'].head()

In [ ]:
df_train['text_deslanged'].head(30)

In [ ]:
#df_train['same'] = df_train['title_and_text'].equals(df_train['title_and_text_deslanged'])
df_train['same'] = np.where(df_train['text'] == df_train['text_deslanged'] , True, False)
df_train['same'].head()

In [ ]:
df_train['same'].value_counts()

## Check regex deslang, correct examples where it is deslanged incorrectly

In [ ]:
df_train.head()

,subreddit,text,label,data_type,text_deslanged,same
0,wallstreetbets,"\n\nAbout Desktop Metal:\n\n* Industry: **Computer Peripherals**\n* Desktop Metal, Inc. engages in the manufacture of additive manufacturing solutions accessible to engineering, design, and manufacturing applications. It operates through the following geographical segments: Americas, Europe Middle East, and Africa, and Asia Pacific. The firm offers 3D printing machines.\n* Market cap 1,55B\n* Price:\~5$\n\nThese days trying to make electric version of everything became a standard and by researching and developing new products you need prototypes. Is there another way to make a prototype than a 3d printer? "" **Quickly 3D print functional prototypes and reduce product development timelines.** "" as they said.\n\nSectors where 3D printing can improve:\n\n* Healthcare 7,38T $\n* Aerospace 1,32T $\n* Automotive 2,62T $\n* Manufacturing 2,13T $\n\nThats just some that come to my mind and just an estimation but can visualize that it can profit from at least 25%(13T $) of the us market(52T...",0,train,"\n\nAbout Desktop Metal:\n\n* Industry: **Computer Peripherals**\n* Desktop Metal, Inc. engages in the manufacture of additive manufacturing solutions accessible to engineering, design, and manufacturing applications. It operates through the following geographical segments: Americas, Europe Middle East, and Africa, and Asia Pacific. The firm offers 3D printing machines.\n* Market cap 1,55B\n* Price:\~5$\n\nThese days trying to make electric version of everything became a standard and by researching and developing new products you need prototypes. Is there another way to make a prototype than a 3d printer? "" **Quickly 3D print functional prototypes and reduce product development timelines.** "" as they said.\n\nSectors where 3D printing can improve:\n\n* Healthcare 7,38T $\n* Aerospace 1,32T $\n* Automotive To,62T $\n* Manufacturing To,13T $\n\nThats just some that come to my mind and just an estimation but can visualize that it can profit from at least 25%(13T $) of the Ultrasound ...",False
2,wallstreetbets,Trust the Chariman,0,train,Trust the Chariman,True
3,wallstreetbets,"$RKT short % of FLOAT has risen to 22% while MSR will increase in value Apes squeeze shorty to the moon. \n\nMSR Mortgage Servicing Rights will increase in value and the cash flow on this float will also increase as rates increase. $RKT is profitable and will be more profitable as MSR increase in value. \n\nRocket Companies Common Stock's NYSE:$RKT short percent of float has risen 21.67% since its last report. The company recently reported that it has 21.84 million shares sold short, which is 17.18% of all regular shares that are available for trading. MSR will increase in Value",0,train,"$RKT short % of FLOAT has risen to 22% while MSR will increase in value Apes squeeze shorty to the moon. \n\nMSR Mortgage Servicing Rights will increase in value and the cash flow on this float will also increase as rates increase. $RKT is profitable and will be more profitable as MSR increase in value. \n\nAttractive person Companies Common Stock's New York Stock Exchange:$RKT short percent of float has risen 21.67% since its last report. The company recently reported that it has 21.84 million shares sold short, which is 17.18% of all regular shares that are available for trading. MSR will increase in Value",False
4,wallstreetbets,"**Read [rules](https://www.reddit.com/r/wallstreetbets/wiki/contentguide), follow [Twitter](https://twitter.com/Official_WSB) and [IG](https://www.instagram.com/official_wallstreetbets/), join [Discord](https://discord.gg/wallstreetbets), see [ban bets](https://www.reddit.com/r/wallstreetbets/wiki/banbets)!**",0,train,"**Read [rules](https://www.reddit.com/Are/wallstreetbets/wiki/contentguide), follow [Twitter](https://twitter.com/Official_WSB) and [Instagram](https://www.instagram.com/official_wallstreetbets/), join [Discord](https://discord.gg/wallstreetbets), see [ban bets](http

In [ ]:
deslanged = df_train[['text', 'text_deslanged']][df_train['same'] == False]

In [ ]:
deslanged_for_bert = df_train[['text_deslanged', 'label']][df_train['same'] == False]

In [ ]:
print(label_dict)

{'wallstreetbets': 0, 'teenagers': 1, 'copypasta': 2, 'GenZ': 3, 'unpopularopinion': 4}


In [ ]:
swapped_dict = {}
swapped_dict = dict([(value, key) for key, value in label_dict.items()])
print(swapped_dict)

{0: 'wallstreetbets', 1: 'teenagers', 2: 'copypasta', 3: 'GenZ', 4: 'unpopularopinion'}


In [ ]:
deslanged_for_bert['subreddit'] = df_train['label'].map(swapped_dict)

In [ ]:
deslanged_for_bert = deslanged_for_bert[['subreddit', 'text_deslanged', 'label']]
deslanged_for_bert = deslanged_for_bert.rename(columns={"text_deslanged": "text"})
deslanged_for_bert.tail()

,subreddit,text,label
2494,unpopularopinion,"I cannot tolerate how over sweetened the fruit preserves are it tastes so much better when I simply Address blueberries or blackberries to my peanut butter sandwich, instead of some random preserved gelatinous goop from that mysterious jar in the back of the fridge.",4
2495,unpopularopinion,"I don’t know what it is but every time I see someone wear slide sandals with Know socks, it always just looks so gross to me. \n\nTheir toes are either hanging over the edge, their feet are dirty, and usually accompanied by gym shorts and oversized tee shirt.",4
2497,unpopularopinion,"When I was about 15, I became cripplingly addicted to porn. I won’t get into too much detail, but it was pretty bad. It lasted for about a year and a half, up until a few months ago when I decided that I couldn’t keep letting myself get away with it, as I had college on the horizon and I would need to dedicate more time to my studies. I’Male an artist and an art student (graphic design specifically), and since I stopped watching porn, I gained a greater appreciation for the human form in a more innocent sense. It is for this reason that I began to enjoy tasteful nudity, both in photos and in art, as well as a few other reasons, the most notable one being that pornography always felt very fake. The actors never felt real to me. Even when I would indulge in porn, I could tell that they were just playing it up for the camera. At the time, I had Know reason to care, but now I do; and if the actors aren’t having a good time, why should I be having a good time? This is what drew me to ta...",4
2498,unpopularopinion,"When I was in deep depression, such messages and campaigns did not help me and they still do not. I imagine this is what it is like for most people when they feel alone and helpless; they want someone that is close to them, someone whom they feel truly cares about them personally, not strangers listening to them on a superficial level. They want friends, not just listeners. I know that some people also just wish to feel understood, but I think many people feel the same I feel.",4
2499,unpopularopinion,"Just to be clear, I am not trying to be insensitive in anyway, and anyone's premature death (as long as they are not some sort of murderer) is tragic. I just find it kind of strange that if a celebrity dies, people all over the world mourns even though they do not know them personally. I think this can sometimes be upsetting for their actual family at times to be constantly reminded over the internet and news that your loved one is gone. Not to mention the conspiracies that sometimes arise and often times it seems like people are more complaining about their deaths than actually mourning which is even worse to see, but while all of that is happening, thousands of other people die without being noticed as if they are so insignificant. Obviously not everyone can be recognized when they die, and I am not saying that everyone should be treated like a celebrity. All I am saying is that it is strange how people act when someone they do not know dies and of course there is Know actual sol...",4


In [ ]:
path = '/content/gdrive/MyDrive/W266/final_project/input_for_bert.csv'

with open(path, 'w', encoding = 'utf-8-sig') as f:
  deslanged_for_bert.to_csv(f)

In [ ]:
deslanged.head()

,text,text_deslanged
0,"\n\nAbout Desktop Metal:\n\n* Industry: **Computer Peripherals**\n* Desktop Metal, Inc. engages in the manufacture of additive manufacturing solutions accessible to engineering, design, and manufacturing applications. It operates through the following geographical segments: Americas, Europe Middle East, and Africa, and Asia Pacific. The firm offers 3D printing machines.\n* Market cap 1,55B\n* Price:\~5$\n\nThese days trying to make electric version of everything became a standard and by researching and developing new products you need prototypes. Is there another way to make a prototype than a 3d printer? "" **Quickly 3D print functional prototypes and reduce product development timelines.** "" as they said.\n\nSectors where 3D printing can improve:\n\n* Healthcare 7,38T $\n* Aerospace 1,32T $\n* Automotive 2,62T $\n* Manufacturing 2,13T $\n\nThats just some that come to my mind and just an estimation but can visualize that it can profit from at least 25%(13T $) of the us market(52T...","\n\nAbout Desktop Metal:\n\n* Industry: **Computer Peripherals**\n* Desktop Metal, Inc. engages in the manufacture of additive manufacturing solutions accessible to engineering, design, and manufacturing applications. It operates through the following geographical segments: Americas, Europe Middle East, and Africa, and Asia Pacific. The firm offers 3D printing machines.\n* Market cap 1,55B\n* Price:\~5$\n\nThese days trying to make electric version of everything became a standard and by researching and developing new products you need prototypes. Is there another way to make a prototype than a 3d printer? "" **Quickly 3D print functional prototypes and reduce product development timelines.** "" as they said.\n\nSectors where 3D printing can improve:\n\n* Healthcare 7,38T $\n* Aerospace 1,32T $\n* Automotive To,62T $\n* Manufacturing To,13T $\n\nThats just some that come to my mind and just an estimation but can visualize that it can profit from at least 25%(13T $) of the Ultrasound ..."
3,"$RKT short % of FLOAT has risen to 22% while MSR will increase in value Apes squeeze shorty to the moon. \n\nMSR Mortgage Servicing Rights will increase in value and the cash flow on this float will also increase as rates increase. $RKT is profitable and will be more profitable as MSR increase in value. \n\nRocket Companies Common Stock's NYSE:$RKT short percent of float has risen 21.67% since its last report. The company recently reported that it has 21.84 million shares sold short, which is 17.18% of all regular shares that are available for trading. MSR will increase in Value","$RKT short % of FLOAT has risen to 22% while MSR will increase in value Apes squeeze shorty to the moon. \n\nMSR Mortgage Servicing Rights will increase in value and the cash flow on this float will also increase as rates increase. $RKT is profitable and will be more profitable as MSR increase in value. \n\nAttractive person Companies Common Stock's New York Stock Exchange:$RKT short percent of float has risen 21.67% since its last report. The company recently reported that it has 21.84 million shares sold short, which is 17.18% of all regular shares that are available for trading. MSR will increase in Value"
4,"**Read [rules](https://www.reddit.com/r/wallstreetbets/wiki/contentguide), follow [Twitter](https://twitter.com/Official_WSB) and [IG](https://www.instagram.com/official_wallstreetbets/), join [Discord](https://discord.gg/wallstreetbets), see [ban bets](https://www.reddit.com/r/wallstreetbets/wiki/banbets)!**","**Read [rules](https://www.reddit.com/Are/wallstreetbets/wiki/contentguide), follow [Twitter](https://twitter.com/Official_WSB) and [Instagram](https://www.instagram.com/official_wallstreetbets/), join [Discord](https://discord.gg/wallstreetbets), see [ban bets](https://www.reddit.com/Are/wallstreetbets/wiki/banbets)!**"
5,"For those who weren't around, /r/place was 3-day event that Reddit had back in 2017. Basically, it starts as a large blan

## T5 Translation

In [ ]:
#model example from: https://simpletransformers.ai/docs/t5-data-formats/

import logging

import pandas as pd
from simpletransformers.t5 import T5Model, T5Args

logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)


train_data = [
    ["binary classification", "Anakin was Luke's father" , "1"],
    ["binary classification", "Luke was a Sith Lord" , "0"],
    ["generate question", "Star Wars is an American epic space-opera media franchise created by George Lucas, which began with the eponymous 1977 film and quickly became a worldwide pop-culture phenomenon", "Who created the Star Wars franchise?"],
    ["generate question", "Anakin was Luke's father" , "Who was Luke's father?"],
]
train_df = pd.DataFrame(train_data)
train_df.columns = ["prefix", "input_text", "target_text"]

eval_data = [
    ["binary classification", "Leia was Luke's sister" , "1"],
    ["binary classification", "Han was a Sith Lord" , "0"],
    ["generate question", "In 2020, the Star Wars franchise's total value was estimated at US$70 billion, and it is currently the fifth-highest-grossing media franchise of all time.", "What is the total value of the Star Wars franchise?"],
    ["generate question", "Leia was Luke's sister" , "Who was Luke's sister?"],
]
eval_df = pd.DataFrame(eval_data)
eval_df.columns = ["prefix", "input_text", "target_text"]

model_args = T5Args()
model_args.num_train_epochs = 10
model_args.no_save = True
model_args.evaluate_generated_text = True
model_args.evaluate_during_training = True
model_args.evaluate_during_training_verbose = True
model_args.overwrite_output_dir = True

model = T5Model("t5", "t5-base", args=model_args, use_cuda=False)


def count_matches(labels, preds):
    print(labels)
    print(preds)
    return sum([1 if label == pred else 0 for label, pred in zip(labels, preds)])


model.train_model(train_df, eval_data=eval_df, matches=count_matches)

print(model.eval_model(eval_df, matches=count_matches))

INFO:simpletransformers.t5.t5_utils: Creating features from dataset file at cache_dir/


  0%|          | 0/4 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:3524: FutureWarning: 
`prepare_seq2seq_batch` is deprecated and will be removed in version 5 of HuggingFace Transformers. Use the regular
`__call__` method to prepare your inputs and the tokenizer under the `as_target_tokenizer` context manager to prepare
your targets.

Here is a short example:

model_inputs = tokenizer(src_texts, ...)
with tokenizer.as_target_tokenizer():
    labels = tokenizer(tgt_texts, ...)
model_inputs["labels"] = labels["input_ids"]

See the documentation of your specific tokenizer for more details on the specific arguments to the tokenizer of choice.
For a more complete example, see the implementation of `prepare_seq2seq_batch`.

  warnings.warn(formatted_warning, FutureWarning)
INFO:simpletransformers.t5.t5_utils: Saving features into cached file cache_dir/t5-base_cached_1284
INFO:simpletransformers.t5.t5_model: Training started


Epoch:   0%|          | 0/10 [00:00<?, ?it/s]

Running Epoch 0 of 10:   0%|          | 0/1 [00:00<?, ?it/s]

...

Running Evaluation:   0%|          | 0/1 [00:00<?, ?it/s]

Generating outputs:   0%|          | 0/1 [00:00<?, ?it/s]

Decoding outputs:   0%|          | 0/4 [00:00<?, ?it/s]

INFO:simpletransformers.t5.t5_model:{'eval_loss': 0.547792911529541, 'matches': 3}


['1', '0', 'What is the total value of the Star Wars franchise?', "Who was Luke's sister?"]
['1', '0', 'Who created the Star Wars franchise?', "Who was Luke's sister?"]
{'eval_loss': 0.547792911529541, 'matches': 3}


In [ ]:
import logging

import pandas as pd
from simpletransformers.t5 import T5Model, T5Args

logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)


train_data = df_train[['text', 'text_deslanged']]
train_data['prefix'] = 'translate'
train_df = train_data[['prefix', 'text', 'text_deslanged']]
train_df = train_df.rename(columns={'prefix': 'prefix', 'text': 'input_text', 'text_deslanged': 'target_text'})

eval_data = df[df.index.isin(X_val)]
eval_data['text_deslanged'] = eval_data['text'].apply(lambda x: slang_lookup(x, slangit_dict))
eval_data = eval_data[['text', 'text_deslanged']]
eval_data['prefix'] = 'translate'
eval_df = eval_data[['prefix', 'text', 'text_deslanged']]
eval_df = eval_df.rename(columns={'prefix': 'prefix', 'text': 'input_text', 'text_deslanged': 'target_text'})

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pyd

In [ ]:
eval_df.head()

,prefix,input_text,target_text
1,translate,"hello, tards. I previously shorted LCID late last year and early this year for some mediocre, paper-handed gains and thinking about doing it again. I look at RIVN and LCID and both seem to have fairly reasonable downfalls aka: don't make many cars/SUVs, don't currently make money, and have gassed up valuations. My discussion question here is, can anyone explain to me the case for an outdoor adventure vehicle that is solely battery-powered? If I am on an adventure through the Grand Canyon or Sierras wouldn't I want the gas backup? Is the idea that battery charging will be ubiquitous in ""the future""?","hello, tards. I previously shorted LCID late last year and early this year for some mediocre, paper-handed gains and thinking about doing it again. I look at RIVN and LCID and both seem to have fairly reasonable downfalls aka: don't make many cars/SUVs, don't currently make money, and have gassed up valuations. My discussion question here is, can anyone explain to me the case for an outdoor adventure vehicle that is solely battery-powered? If I am on an adventure through the A thousand dollars Canyon or Sierras wouldn't I want the gas backup? Is the idea that battery charging will be ubiquitous in ""the future""?"
13,translate,"\n\nThe recent recovery in stocks may be short-lived, according to Bank of America Securities, which sees a bear market setting in through 2022 and suggests that investors tilt to cash and commodities.\n\nBofA laid out a witches’ brew of negative trends, signals, and data points in a note published Thursday by chief investment strategist Michael Hartnett and his team. “If it walks like a bear…” it probably is, they write.\n\nMore than 75% of stocks in the [Nasdaq Composite ](https://www.barrons.com/market-data/indexes/comp)Index and 51% of [S&amp;P 500](https://www.barrons.com/market-data/indexes/spx) stocks are already in a bear market—down more than 20% from peak prices, Hartnett notes. The outlook is worsening with geopolitical risks exacerbating potential for inflation, higher commodity prices, and “shocks” to growth.\n\nNegative real interest rates, adjusted for inflation, are another ominous sign. Going back 250 years, negative rates have been “synonymous” with crashes, pani...","\n\nThe recent recovery in stocks may be short-lived, according to Bank of America Securities, which sees a bear market setting in through 2022 and suggests that investors tilt to cash and commodities.\n\nBofA laid out a witches’ brew of negative trends, signals, and data points in a note published Thursday by chief investment strategist Michael Hartnett and his team. “If it walks like a bear…” it probably is, they write.\n\nMore than 75% of stocks in the [Nasdaq Composite ](https://www.barrons.com/market-data/indexes/Computer)Index and 51% of [S&To be loud and angry;P 500](https://www.barrons.com/market-data/indexes/spx) stocks are already in a bear market—down more than 20% from peak prices, Hartnett notes. The outlook is worsening with geopolitical risks exacerbating potential for inflation, higher commodity prices, and “shocks” to growth.\n\nNegative real interest rates, adjusted for inflation, are another ominous sign. Going back 250 years, negative rates have been “synonymou..."
20,translate,The sudden but inevitable betrayal,The sudden but inevitable betrayal
21,translate,You win this round Bulls,You win this round Bulls
34,translate,War officially over. Jim Cramer for Nobel Peace Prize.,War officially over. Jim Cramer for Nobel Peace Prize.


In [ ]:
model_args = T5Args()
model_args.num_train_epochs = 10
model_args.no_save = True
model_args.evaluate_generated_text = True
model_args.evaluate_during_training = True
model_args.evaluate_during_training_verbose = True
model_args.overwrite_output_dir = True
torch.cuda.memory_summary(device=None, abbreviated=False)
model_args.per_gpu_train_batch_size = 128

model = T5Model("t5", "t5-base", args=model_args, use_cuda=True)


In [ ]:
def count_matches(labels, preds):
    print(labels)
    print(preds)
    return sum([1 if label == pred else 0 for label, pred in zip(labels, preds)])

In [ ]:
torch.cuda.memory_summary(device=None, abbreviated=False)

'|===========================================================================|\n|                  PyTorch CUDA memory summary, device ID 0                 |\n|---------------------------------------------------------------------------|\n|            CUDA OOMs: 0            |        cudaMalloc retries: 0         |\n|===========================================================================|\n|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |\n|---------------------------------------------------------------------------|\n| Allocated memory      |       0 B  |       0 B  |       0 B  |       0 B  |\n|       from large pool |       0 B  |       0 B  |       0 B  |       0 B  |\n|       from small pool |       0 B  |       0 B  |       0 B  |       0 B  |\n|---------------------------------------------------------------------------|\n| Active memory         |       0 B  |       0 B  |       0 B  |       0 B  |\n|       from large pool |       0 B  |       0 B  |

In [ ]:
torch.cuda.empty_cache()

In [ ]:
model.train_model(train_df, eval_data=eval_df, matches=count_matches)

INFO:simpletransformers.t5.t5_utils: Creating features from dataset file at cache_dir/


  0%|          | 0/2000 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:3524: FutureWarning: 
`prepare_seq2seq_batch` is deprecated and will be removed in version 5 of HuggingFace Transformers. Use the regular
`__call__` method to prepare your inputs and the tokenizer under the `as_target_tokenizer` context manager to prepare
your targets.

Here is a short example:

model_inputs = tokenizer(src_texts, ...)
with tokenizer.as_target_tokenizer():
    labels = tokenizer(tgt_texts, ...)
model_inputs["labels"] = labels["input_ids"]

See the documentation of your specific tokenizer for more details on the specific arguments to the tokenizer of choice.
For a more complete example, see the implementation of `prepare_seq2seq_batch`.

  warnings.warn(formatted_warning, FutureWarning)
INFO:simpletransformers.t5.t5_utils: Saving features into cached file cache_dir/t5-base_cached_1282000
INFO:simpletransformers.t5.t5_model: Training started


Epoch:   0%|          | 0/10 [00:00<?, ?it/s]

Running Epoch 0 of 10:   0%|          | 0/250 [00:00<?, ?it/s]

KeyboardInterrupt: ignored

In [ ]:
my_t5 = model.eval_model(eval_df, matches=count_matches)
print(my_t5)

In [ ]:
print(my_t5)

In [ ]:
#get predictions included in df
preds = model.predict(list(eval_df['input_text']))

In [ ]:
print(preds)

In [ ]:
eval_df['t5_prediction'] = preds
eval_df.head()

In [ ]:
eval_df.to_csv('reddit_eval_t5_translated.csv')